In [ ]:
# ============================================================================================
# PROBLEM B5
#
# Build and train a neural network model using the Daily Max Temperature.csv dataset.
# Use MAE as the metrics of your neural network model.
# We provided code for normalizing the data. Please do not change the code.
# Do not use lambda layers in your model.
#
# The dataset used in this problem is downloaded from https://github.com/jbrownlee/Datasets
#
# Desired MAE < 0.2 on the normalized dataset.
# ============================================================================================

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv
import urllib
import pandas as pd
from tensorflow.keras.optimizers import Adam


def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)


def solution_B5():
    df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-max-temperatures.csv')
    series = df["Temperature"]
    # Normalization Function. DO NOT CHANGE THIS CODE
    min=np.min(series)
    max=np.max(series)
    series -= min
    series /= max
   # time=np.array(time_step)

    # DO NOT CHANGE THIS CODE
    split_time=2500

    x_train=series[:2500]
    x_valid= series[2500:]

    # DO NOT CHANGE THIS CODE
    window_size=64
    batch_size=256
    shuffle_buffer_size=1000

    train_set=windowed_dataset(
        x_train, window_size, batch_size, shuffle_buffer_size)
    print(train_set)
    print(x_train.shape)
    valid_set = windowed_dataset(x_valid,window_size,batch_size,shuffle_buffer_size)

    model=tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(16,3, strides =1,
                                padding = "causal",
                               activation = "relu",
                               input_shape = [None,1]),
        tf.keras.layers.LSTM(64,return_sequences = True),
        tf.keras.layers.LSTM(64,return_sequences = True),
        tf.keras.layers.LSTM(128,return_sequences = True),
        tf.keras.layers.Dense(64,activation = "relu"),
        tf.keras.layers.Dense(128,activation = "relu"),
        tf.keras.layers.Dense(1),
    ])

    lr_schedule = tf.keras.callbacks.LearningRateScheduler(
      lambda epoch: 1e-8 * 10**(epoch / 20))

    model.compile(loss = tf.keras.losses.Huber(),
                  optimizer = Adam(learning_rate = 0.0001),
                  metrics =["mae"])
    #model.fit(train_set,epochs = 100,verbose = 1 ,callbacks = [lr_schedule])
    # YOUR CODE HERE

    model.compile(loss = tf.keras.losses.Huber(),
                  optimizer =Adam(learning_rate = 0.001),
                  metrics = ["mae"])
    model.fit(train_set,
              epochs = 20,
              verbose = 1 ,
              validation_data = valid_set)
    return model
# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model=solution_B5()
    model.save("model_B5.h5")

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None))>
(2500,)
Epoch 1/20
10/10 [==============================] - 15s 908ms/step - loss: 0.0270 - mae: 0.1890 - val_loss: 0.0104 - val_mae: 0.1028
Epoch 2/20
10/10 [==============================] - 8s 773ms/step - loss: 0.0098 - mae: 0.1012 - val_loss: 0.0086 - val_mae: 0.1063
Epoch 3/20
10/10 [==============================] - 8s 774ms/step - loss: 0.0083 - mae: 0.1005 - val_loss: 0.0085 - val_mae: 0.0933
Epoch 4/20
10/10 [==============================] - 8s 772ms/step - loss: 0.0077 - mae: 0.0962 - val_loss: 0.0073 - val_mae: 0.0900
Epoch 5/20
10/10 [==============================] - 8s 773ms/step - loss: 0.0069 - mae: 0.0898 - val_loss: 0.0064 - val_mae: 0.0833
Epoch 6/20
10/10 [==============================] - 8s 773ms/step - loss: 0.0061 - mae: 0.0839 - val_loss: 0.0061 - val_mae: 0.0836
Epoch 7/20
10/10 [=====================